## Chapter 6: Model Validation

In the previous chapter, you learned how to build forms and accept user input. However, accepting input is only half the battle—you must also ensure that the data is valid, secure, and meaningful. Model validation is the process of checking that the data submitted by users meets your application's requirements before you process it. ASP.NET Core provides a rich, extensible validation framework that works both on the server and the client. In this chapter, you'll master server-side validation with data annotations, create custom validation attributes, implement client-side validation for immediate feedback, handle validation errors gracefully in controllers, and explore advanced scenarios like remote validation. By the end, you'll be able to build robust forms that protect your application from invalid or malicious data.

### 6.1 Server-Side Validation with Data Annotations

Server-side validation is the foundation of data integrity. Even if you have client-side validation, you must always validate on the server because client-side checks can be bypassed. ASP.NET Core integrates seamlessly with **data annotations**—attributes you apply to model properties to define validation rules.

#### Common Validation Attributes

Here are the most frequently used validation attributes:

| Attribute | Description | Example |
|-----------|-------------|---------|
| `[Required]` | Ensures the property is not null or empty. For value types (like `int`), you may need to make them nullable (`int?`) if you want to distinguish between missing and zero. | `[Required(ErrorMessage = "Name is required.")]` |
| `[StringLength(max)]` | Limits the length of a string. Can specify `MinimumLength`. | `[StringLength(100, MinimumLength = 3)]` |
| `[Range(min, max)]` | Restricts a numeric value to a specified range. | `[Range(0.01, 10000.00)]` |
| `[EmailAddress]` | Validates that the string is in a valid email format. | `[EmailAddress]` |
| `[Phone]` | Validates a phone number format (loose, based on locale). | `[Phone]` |
| `[Url]` | Validates a URL format. | `[Url]` |
| `[Compare]` | Ensures two properties match (e.g., password confirmation). | `[Compare("Password")]` |
| `[RegularExpression]` | Matches the property against a regular expression. | `[RegularExpression(@"^[A-Z]+[a-zA-Z]*$")]` |
| `[CreditCard]` | Validates a credit card number format (Luhn algorithm). | `[CreditCard]` |

**Applying attributes to a model:**

Let's enhance our `Product` model with thorough validation:

```csharp
using System.ComponentModel.DataAnnotations;

public class Product
{
    public int Id { get; set; }

    [Required(ErrorMessage = "Product name is required.")]
    [StringLength(100, MinimumLength = 3, ErrorMessage = "Name must be between 3 and 100 characters.")]
    [Display(Name = "Product Name")]
    public string Name { get; set; }

    [Required(ErrorMessage = "Price is required.")]
    [Range(0.01, 10000.00, ErrorMessage = "Price must be between 0.01 and 10000.00.")]
    [DataType(DataType.Currency)]
    public decimal Price { get; set; }

    [StringLength(500, ErrorMessage = "Description cannot exceed 500 characters.")]
    [DataType(DataType.MultilineText)]
    public string Description { get; set; }

    [Required(ErrorMessage = "Please select a category.")]
    [Display(Name = "Category")]
    public int CategoryId { get; set; }

    [Display(Name = "Available for purchase?")]
    public bool IsAvailable { get; set; }

    [Url(ErrorMessage = "Please enter a valid URL.")]
    [Display(Name = "Product Image URL")]
    public string ImageUrl { get; set; }
}
```

**How validation works:**

When a request arrives, the model binding system populates the model. After binding, the validation system checks each property against the applied attributes. Any violations are recorded in `ModelState`, a dictionary that tracks both the submitted values and any errors.

#### Validation in Controllers

In your POST action, you check `ModelState.IsValid`:

```csharp
[HttpPost]
[ValidateAntiForgeryToken]
public IActionResult Create(Product product)
{
    if (ModelState.IsValid)
    {
        // Save to database
        _productService.Add(product);
        TempData["SuccessMessage"] = "Product created!";
        return RedirectToAction(nameof(Index));
    }

    // If invalid, redisplay the form with errors
    // Repopulate any dropdown data
    ViewBag.CategoryList = GetCategorySelectList();
    return View(product);
}
```

If `ModelState` is invalid, you return the view with the model. The Tag Helpers (`<span asp-validation-for>`) will display the error messages.

#### Validation for Complex Types and Nested Objects

Validation attributes can also be applied to properties that are themselves complex objects. ASP.NET Core will recursively validate those objects.

```csharp
public class Order
{
    [Required]
    public Customer Customer { get; set; }
    
    [ValidateComplexType] // Optional, tells validator to validate this property's properties
    public List<OrderItem> Items { get; set; }
}

public class Customer
{
    [Required]
    public string Name { get; set; }
    
    [EmailAddress]
    public string Email { get; set; }
}
```

By default, complex type properties are validated if they have validation attributes. Collections are also validated recursively. You can also use the `[ValidateComplexType]` attribute to enforce validation even if the property itself doesn't have attributes.

#### Validation in Web APIs

For Web API controllers (decorated with `[ApiController]`), invalid `ModelState` automatically triggers a 400 Bad Request response with details. You don't need to check `ModelState.IsValid` manually—the framework does it for you.

```csharp
[ApiController]
[Route("api/[controller]")]
public class ProductsApiController : ControllerBase
{
    [HttpPost]
    public IActionResult Create(Product product)
    {
        // If model is invalid, the framework returns 400 automatically
        // You only reach this code if ModelState.IsValid is true
        _productService.Add(product);
        return CreatedAtAction(nameof(Get), new { id = product.Id }, product);
    }
}
```

You can customize the automatic response by modifying the `ApiBehaviorOptions` in `Program.cs`:

```csharp
builder.Services.Configure<ApiBehaviorOptions>(options =>
{
    options.InvalidModelStateResponseFactory = context =>
    {
        var problems = new CustomErrorPayload(context.ModelState);
        return new BadRequestObjectResult(problems);
    };
});
```

---

### 6.2 Custom Validation Attributes

Sometimes built-in attributes aren't enough. You might need to validate against business rules, such as ensuring a product's release date is not in the past, or that a discount percentage doesn't exceed 50%. You can create **custom validation attributes** by deriving from `ValidationAttribute` and overriding the `IsValid` method.

#### Simple Custom Validator

Let's create an attribute that ensures a date is in the future:

```csharp
using System.ComponentModel.DataAnnotations;

public class FutureDateAttribute : ValidationAttribute
{
    public override bool IsValid(object value)
    {
        if (value is DateTime date)
        {
            return date > DateTime.Now;
        }
        return true; // Returning true for null values; you can combine with [Required] if needed
    }
}
```

Apply it to a property:

```csharp
[FutureDate(ErrorMessage = "Release date must be in the future.")]
[DataType(DataType.Date)]
public DateTime ReleaseDate { get; set; }
```

#### Accessing the Entire Object for Validation

Sometimes validation depends on multiple properties. For example, a product's discount should not exceed its price. For such cross-property validation, you can override `IsValid` with a `ValidationContext` parameter that gives you access to the object instance.

```csharp
public class DiscountNotExceedingPriceAttribute : ValidationAttribute
{
    protected override ValidationResult IsValid(object value, ValidationContext validationContext)
    {
        var product = (Product)validationContext.ObjectInstance;
        
        if (product.Discount > product.Price)
        {
            return new ValidationResult("Discount cannot exceed the price.");
        }
        
        return ValidationResult.Success;
    }
}
```

Apply it at the class level:

```csharp
[DiscountNotExceedingPrice]
public class Product
{
    public decimal Price { get; set; }
    public decimal Discount { get; set; }
}
```

#### Using Dependency Injection in Custom Attributes

Custom validation attributes are instantiated by the framework, so they don't support dependency injection directly. However, you can access services through the `ValidationContext` if you register a service provider. This is useful for validation that requires database access, like checking uniqueness.

```csharp
protected override ValidationResult IsValid(object value, ValidationContext validationContext)
{
    var service = (IMyService)validationContext.GetService(typeof(IMyService));
    // use service to validate
}
```

For this to work, the service must be registered in DI, and the validation context must have access to the service provider (which it does by default).

**Note:** Be cautious with database access during validation—it can impact performance and lead to unexpected side effects.

---

### 6.3 Client-Side Validation

Client-side validation provides immediate feedback to users without a round trip to the server. It's implemented using jQuery Validation and jQuery Unobtrusive Validation, which are included in the default ASP.NET Core MVC template.

#### How It Works

When you use Tag Helpers like `<input asp-for="Name">`, they generate HTML5 `data-*` attributes based on your validation attributes:

```html
<input 
  type="text" 
  data-val="true" 
  data-val-required="The Name field is required." 
  data-val-length="The field Name must be a string with a maximum length of 100." 
  data-val-length-max="100" 
  id="Name" 
  name="Name" 
  value="" />
```

The jQuery Unobtrusive Validation library reads these attributes and sets up jQuery Validation rules automatically.

#### Enabling Client-Side Validation

In your views or layout, you need to include the validation scripts. Typically, you add this to the bottom of your view (or in `_Layout.cshtml` if you want it on every page):

```html
@section Scripts {
    <partial name="_ValidationScriptsPartial" />
}
```

The `_ValidationScriptsPartial` (located in `Views/Shared`) contains references to:

```html
<script src="~/lib/jquery/dist/jquery.min.js"></script>
<script src="~/lib/jquery-validation/dist/jquery.validate.min.js"></script>
<script src="~/lib/jquery-validation-unobtrusive/jquery.validate.unobtrusive.min.js"></script>
```

Once included, client-side validation is automatic. Users will see error messages as they type or when they submit the form, without a page refresh.

#### Customizing Client-Side Validation

You can create custom client-side validation adapters to mirror your custom server-side attributes. This involves writing JavaScript that integrates with jQuery Validation. For example, to support our `FutureDateAttribute`, we would need to:

1. Add a `data` attribute to the input that our custom validator can recognize. This is done by implementing `IClientModelValidator` on the attribute.
2. Write JavaScript that registers a new validation method.

**Server-side attribute implementing `IClientModelValidator`:**

```csharp
public class FutureDateAttribute : ValidationAttribute, IClientModelValidator
{
    public void AddValidation(ClientModelValidationContext context)
    {
        context.Attributes.Add("data-val", "true");
        context.Attributes.Add("data-val-futuredate", ErrorMessage ?? "Date must be in the future.");
    }

    public override bool IsValid(object value)
    {
        // server-side validation as before
    }
}
```

**JavaScript (to be included in your view or a script file):**

```javascript
$.validator.addMethod("futuredate", function (value, element) {
    var date = new Date(value);
    return isNaN(date) || date > new Date();
});

$.validator.unobtrusive.adapters.addBool("futuredate");
```

Now your custom validation works on both client and server.

#### When to Rely on Client-Side Validation

Client-side validation enhances user experience but is not a security measure. Always validate on the server. Never trust client-side validation alone.

---

### 6.4 Handling Invalid Model State in Controllers

When `ModelState` is invalid, you need to redisplay the form with errors and the user's submitted data. This ensures they can correct mistakes without retyping everything.

#### Repopulating Dropdowns and Other UI Data

Remember that data used to populate dropdowns (like `ViewBag.CategoryList`) is not part of `ModelState` and must be recreated in the POST action when returning the view.

```csharp
[HttpPost]
[ValidateAntiForgeryToken]
public IActionResult Create(Product product)
{
    if (ModelState.IsValid)
    {
        // success
        return RedirectToAction(nameof(Index));
    }

    // Repopulate dropdowns
    ViewBag.CategoryList = GetCategorySelectList();
    return View(product);
}
```

#### Adding Custom Errors to ModelState

Sometimes you have business logic validation that doesn't fit neatly into a property attribute. You can add errors directly to `ModelState`:

```csharp
if (product.Price > 10000 && string.IsNullOrEmpty(product.ApprovalCode))
{
    ModelState.AddModelError(nameof(product.ApprovalCode), 
        "Approval code is required for high-value products.");
}

// Or add a general error not tied to a specific property
ModelState.AddModelError(string.Empty, "An error occurred processing your request.");
```

The error tied to a specific property will display next to that field's validation message span. The general error (empty key) can be displayed using `Html.ValidationSummary()`.

#### Validation Summary

The Validation Summary Tag Helper displays a summary of all validation errors:

```html
<div asp-validation-summary="All" class="text-danger"></div>
```

You can specify:
- `All` – shows all errors (both property-level and model-level).
- `ModelOnly` – shows only model-level errors (those with empty key).
- `None` – shows nothing (useful if you want to suppress the summary).

#### Preserving User Input

One of the benefits of returning the model instance to the view is that Tag Helpers will repopulate the input fields with the submitted values (stored in `ModelState`). This happens automatically because the `asp-for` Tag Helpers use `ModelState` to retrieve the attempted value.

For example, if the user enters an invalid price, when the form redisplays, the input will still contain their invalid entry (so they can correct it), not the original model's default value.

---

### 6.5 Remote Validation

Sometimes you need to validate a value against data on the server without submitting the entire form—for example, checking if a username is already taken. **Remote validation** enables this: it makes an AJAX call to a server action to validate a single field.

#### Setting Up Remote Validation

1. Add a `[Remote]` attribute to the property, specifying the controller and action to call.

```csharp
public class UserRegistrationViewModel
{
    [Required]
    [Remote(action: "VerifyUsername", controller: "Account")]
    public string Username { get; set; }
}
```

2. Implement the action in the specified controller. It should return a `JsonResult` indicating whether the value is valid.

```csharp
public class AccountController : Controller
{
    [AcceptVerbs("GET", "POST")]
    public IActionResult VerifyUsername(string username)
    {
        // Check if username exists (e.g., in database)
        bool usernameTaken = CheckIfUsernameExists(username);
        
        if (usernameTaken)
        {
            return Json($"Username {username} is already taken.");
        }
        
        return Json(true); // indicates success
    }
}
```

The action can return:
- `true` (as JSON) if validation passes.
- A string error message if validation fails.
- `false` if you want a default error message.

#### Customizing Remote Validation

You can specify additional fields to send to the server using the `AdditionalFields` parameter:

```csharp
[Remote(action: "VerifyEmail", controller: "Account", AdditionalFields = "Username")]
public string Email { get; set; }
```

The server action can then accept both parameters:

```csharp
public IActionResult VerifyEmail(string email, string username)
```

#### Client-Side Integration

The `[Remote]` attribute adds `data-val-remote-*` attributes to the input, which the unobtrusive validation library uses to make the AJAX call. The user sees the error message returned from the server.

**Important:** Remote validation is a performance consideration—it fires on every keystroke by default. You can debounce it or adjust the event that triggers validation, but that's done via JavaScript customization.

---

### 6.6 Validation in Web APIs

As mentioned, Web API controllers with `[ApiController]` automatically return a 400 response with a `ProblemDetails` object containing validation errors. The default response structure is:

```json
{
  "type": "https://tools.ietf.org/html/rfc7231#section-6.5.1",
  "title": "One or more validation errors occurred.",
  "status": 400,
  "traceId": "00-123...",
  "errors": {
    "Name": ["The Name field is required."],
    "Price": ["Price must be between 0.01 and 10000.00."]
  }
}
```

This is standard and works well with client-side frameworks.

#### Customizing API Validation Responses

If you need a different format, you can configure the `ApiBehaviorOptions` as shown earlier. You can also disable automatic 400 responses and handle validation manually by removing the `[ApiController]` attribute or by setting `SuppressModelStateInvalidFilter` to `true`.

```csharp
builder.Services.Configure<ApiBehaviorOptions>(options =>
{
    options.SuppressModelStateInvalidFilter = true;
});
```

Then you must check `ModelState.IsValid` manually and return appropriate responses.

#### Validation in Minimal APIs

For minimal APIs, validation is not automatic. You must manually check `ModelState` (which isn't available) or use libraries like `FluentValidation` to validate incoming data. For example:

```csharp
app.MapPost("/products", async (Product product, MyDbContext db) =>
{
    var validator = new ProductValidator();
    var result = await validator.ValidateAsync(product);
    if (!result.IsValid)
    {
        return Results.ValidationProblem(result.ToDictionary());
    }
    
    db.Products.Add(product);
    await db.SaveChangesAsync();
    return Results.Created($"/products/{product.Id}", product);
});
```

---

### 6.7 Best Practices and Security Considerations

1. **Always validate on the server.** Client-side validation is for user experience only.
2. **Use view models, not domain entities.** Bind to a view model that contains only the properties you expect from the client. This prevents over-posting attacks where a malicious user adds extra fields to the request to modify properties they shouldn't (e.g., setting `IsAdmin` to true).
3. **Keep validation logic in one place.** Ideally, use data annotations or a dedicated validation library like FluentValidation to centralize rules.
4. **Provide clear error messages.** Tell users what went wrong and how to fix it.
5. **Sanitize and encode output.** When redisplaying user input (e.g., in a textbox), Tag Helpers automatically encode it, preventing XSS attacks.
6. **Be cautious with remote validation.** Ensure the validation action is secured (e.g., requires authentication) if it exposes sensitive data.
7. **Consider performance.** Complex validation (e.g., database lookups) can be expensive; use caching where appropriate.

---

### Summary

You've now mastered model validation in ASP.NET Core:
- **Server-side validation** with data annotations ensures data integrity.
- **Custom validation attributes** handle business-specific rules.
- **Client-side validation** improves user experience with immediate feedback.
- **Handling invalid ModelState** in controllers lets you redisplay forms with errors and user input.
- **Remote validation** provides real-time server checks like uniqueness.
- **API validation** integrates seamlessly with automatic 400 responses.

With these tools, you can build forms and APIs that reject invalid data gracefully and securely.

**Exercise:**
1. Extend the `Product` model from Chapter 5 with additional validation: `ReleaseDate` (future date), `SKU` (required, alphanumeric, exactly 10 characters using `[RegularExpression]`).
2. Create a custom validation attribute `[NonEmptyList]` for a collection property (e.g., `List<string> Tags`) that ensures the list is not empty.
3. Add client-side support for this custom attribute.
4. In the `ProductsController.Create` POST action, add a business rule: if `Price > 1000`, then `ApprovalCode` is required. Add this error to `ModelState` and display it in the view.
5. Implement remote validation to check if a product SKU is already taken (simulate with a static list). Make sure the validation fires on the client.

In the next chapter, **"Dependency Injection and The Repository Pattern,"** you'll learn how to write loosely coupled, testable code by leveraging ASP.NET Core's built-in DI container. You'll implement the Repository pattern to abstract data access and see how DI promotes maintainability and testability in real-world applications.